In [1]:
import sys
sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\guidance-pythonic\\guidance')

import guidance

In [2]:
from environs import Env
import os

env = Env()
env.read_env('./.env')

# DONT FORGET TO CREATE THE .env file 
os.environ["OPENAI_API_TYPE"] = env("OPENAI_API_TYPE")
# os.environ["OPENAI_API_VERSION"] = env("OPENAI_API_VERSION")
# os.environ["OPENAI_API_BASE"] = env("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = env("OPENAI_API_KEY")

In [4]:
lm = guidance.models.ChatOpenAI("gpt-4")

lm = lm.system("You are a helpful and terse assistant.")

with lm.user() as lm:
        lm += f"""Please tell me what the difference is between a frog and a toad."""

with lm.assistant() as lm:
        #lm = lm.gen('answer', temperature=0, max_tokens=50)
        lm = lm.select('answer', ["toads are better", "frogs are better"])
        #lm = lm.select('answer', ["toads are better", "frogs are better", "frogs are worse"])
        ## next line throws an error.
        #lm = lm.select('answer', ["frogs are better", "frogs are worse than toads"])

lm

In [4]:
lm = guidance.models.Transformers("gpt2", temperature=1.0)

lm = lm("You are a helpful and terse assistant.\n")

lm += f"""Please tell me what the difference is between a frog and a toad. """

#lm = lm.gen('answer', temperature=0, max_tokens=50)
lm = lm.select('answer', ["Toads are better", "Frogs are better", "Frogs are worse"])
#lm = lm.select('answer', ["toads are better", "frogs are better", "frogs are worse"])
## next line throws an error.
#lm = lm.select('answer', ["frogs are better", "frogs are worse than toads"])


In [14]:
lm._variables

{'answer': 'Frogs are better'}

In [6]:
class Constraint:

    def __init__(self):
        pass

    def set_context(self, prefix, lm):
        pass

    # TODO: switch interface to return next_char and make the shared code convert to tokens.
    def valid_next_chars(self):  ## can return nothing if generation is complete
        pass

    def set_next_chars(self, c): ## returns a new immutable Constraint object with an updated internal state
        return self

class DummyCharacterConstraint(Constraint):

    def __init__(self, positiveConstraint = ['a','b','c']):
        self.positiveConstraint = positiveConstraint
        #self.negativeConstraint = negativeConstraint
    
    def valid_next_chars(self):
        return self.positiveConstraint


In [2]:
lm = guidance.models.Transformers("gpt2", temperature=1.0)

C:\Users\emrek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
def recursive_getnext_valid_token(constraint, validstr=""):
    ret = []
    next_valid_chars = constraint.valid_next_chars()
    if( len(next_valid_chars) > 0):
        for c in next_valid_chars:
            valid_tokens = lm.get_encoded(str(validstr + c))
            if( len(valid_tokens) > 1):
                #print(f"valid token: {valid_tokens[0]}")
                ret.append(valid_tokens[0])
            else:
                # TODO -- need to update constraint with 'c' so that it has the right context
                next_constraint = constraint.set_next_char(c)
                ret.extend(recursive_getnext_valid_token(next_constraint, validstr + c))                
        return set(ret)
    else:
        # TODO pull characters from suffix.
        return []



In [43]:

next_valid_tokens = recursive_getnext_valid_token(DummyCharacterConstraint(positiveConstraint=['a', 'b']))

for t in next_valid_tokens:
    print(f"{t} = {lm.get_decoded([t])}")


valid token: 24794
valid token: 46071
valid token: 7252
valid token: 64
valid token: 397
valid token: 397
valid token: 6485
valid token: 6485
valid token: 6485
valid token: 7012
valid token: 65
valid token: 65
valid token: 11848
64 = a
65 = b
7012 = ba
11848 = bb
397 = ab
7252 = aa
6485 = abb
46071 = aaa
24794 = aaaa


In [43]:
def constrain(lm, name="value", constraint=None, suffix="", logprobs=None, list_append=False):
    ''' Select a value from a list of choices.

    Parameters
    ----------
    name : str
        The name of the variable to set with the value generated under constraints.
    constraint : Constraint
        A Constraint object that .
    suffix : str
        An optional suffix to append to the selected value. Passing the next string as a suffix allows the select
        statement to better differentiate between options that depend on the final token (and hence on a token that may overlap the following text).
    list_append : bool
        Whether to append the generated value to a list stored in the variable. If set to True, the variable
        must be a list, and the generated value will be appended to the list.
    '''
    
    assert constraint is not None, "You must provide a constraint!"

    ### EMK: We will give the suffix to the constraint provider when asking for the next token
    ###      When the constraint provider reaches the end of an option/program/etc, it will append the suffix    
    def recursive_getnext_valid_token(constraint, validstr=""):
        ret = []
        next_valid_chars = constraint.valid_next_chars()
        if( len(next_valid_chars) > 0):
            for c in next_valid_chars:
                valid_tokens = lm.get_encoded(str(validstr + c))
                if( len(valid_tokens) > 1):
                    #print(f"valid token: {valid_tokens[0]}")
                    ret.append(valid_tokens[0])
                else:
                    next_constraint = constraint.set_next_chars(c)
                    ret.extend(recursive_getnext_valid_token(next_constraint, validstr + c))                
            return set(ret)
        else:
            # TODO pull characters from suffix.
            return []

    def gen_next_token(current_prefix, valid_next_tokens):
        logit_bias = {}
        for token in valid_next_tokens:
            logit_bias[token] = 100

        print(f"Current prefix = {current_prefix}")
        gen_obj = lm.get_endpoint_session()(
            current_prefix, # TODO: perhaps we should allow passing of token ids directly? (this could allow us to avoid retokenizing the whole prefix many times)
            max_tokens=1,
            logit_bias=logit_bias,
            logprobs=len(logit_bias),
            cache_seed=0,
            stream=False,
            token_healing=False # we manage token boundary healing ourselves for this function
        )
        gen_obj = gen_obj["choices"][0] # get the first choice (we only asked for one)
        print(gen_obj)
        return gen_obj["text"] # TODO extend this to return the gen_obj

    gen = str(lm)
    current_constraint = constraint

    # TODO extend this to do a beam search
    for i in range(10):
        next_valid_tokens = recursive_getnext_valid_token(current_constraint)
        if( len(next_valid_tokens) == 0):
            break

        next_token = gen_next_token(gen, next_valid_tokens) 
        next_chars = next_token # the token is returned as characters, not as a token id
        gen += next_chars
        current_constraint = current_constraint.set_next_chars(next_chars)

    return gen

In [7]:
lm = guidance.models.Transformers("gpt2", temperature=1.0)

C:\Users\emrek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lm("Hello").gen('next', max_tokens=10)

In [4]:
from guidance import constrain

In [5]:
from guidance.library import DummyCharacterConstraint


In [8]:
lm = lm("Hello").constrain(name = "var1", constraint=DummyCharacterConstraint(positiveConstraint=[' ', ',', 'W', 'w', 'o', 'r', 'l', 'd', '!'])) # 'I', '\'', 'm', 'p', 'g', 'a', 'm', 'B', 'o', 'b', 

In [9]:
lm['var1']

'Hello, Wow, Wow! Woooo, Wow!'

In [5]:
from guidance.library import OddEvenDigitConstraint

lm = guidance.models.Transformers("gpt2", temperature=1.0)

lm = lm("Hello").constrain(name = "var1", constraint=OddEvenDigitConstraint(), max_tokens=10)

In [2]:
import sys
sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\guidance-pythonic\\guidance')
import guidance
from guidance.library.constraints import ContextFreeGrammarConstraint


ImportError: cannot import name 'ContextFreeGrammarConstraint' from 'guidance.library.constraints' (c:\users\emrek\source\guidance\guidance-pythonic\guidance\guidance\library\constraints\__init__.py)

In [6]:
from guidance.library import Constraint


In [1]:

lm = guidance.models.Transformers("gpt2", temperature=1.0)

testCFG = ContextFreeGrammarConstraint()
testCFG.add_grammar_rule(testCFG.Rule("S", ["@i", "@f", "@ ", "E", "@ ", "@t", "@h", "@e", "@n", "@ ", "C"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@1"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@2"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@3"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@4"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@5"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@6"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@7"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@8"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@9"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["@(", "E", "@)"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@+", "E"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@-", "E"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@*", "E"]))
testCFG.add_grammar_rule(testCFG.Rule("E", ["E", "@/", "E"]))
testCFG.add_grammar_rule(testCFG.Rule("C", ["@X", "@=", "E"]))
testCFG.finalize_grammar()

lm = lm("Hello").constrain(name = "var1", constraint=testCFG, max_tokens=10)



In [6]:
init_lm = guidance.models.Transformers("gpt2", temperature=1.0)

In [7]:
lm = init_lm("If 2+4 then return 8: ").constrain(name = "var1", constraint=testCFG, max_tokens=10)


In [8]:
rules = testCFG.mtable[('E', '1')]
for r in rules:
    print(f"{r.symbol} -> {r.productions}")

E -> ['@1']
